# Interactive Remote Sensing Dashboard with Streamlit

This notebook demonstrates how to create an interactive dashboard for remote sensing data analysis using `Streamlit`, `rasterio`, `geopandas`, `folium`, and `numpy` in Python. Users can upload a Sentinel-2 GeoTIFF and a vector file (e.g., GeoJSON), calculate indices like NDVI or NDWI, and visualize results on an interactive map. The dashboard is designed to be user-friendly for non-technical users.

## Prerequisites
- Install required libraries: `streamlit`, `rasterio`, `geopandas`, `folium`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- A preprocessed Sentinel-2 GeoTIFF (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A GeoJSON or shapefile defining an area of interest (AOI) or labels (e.g., `aoi.geojson`).
- Run this notebook as a Streamlit app: `streamlit run 33_dashboard_streamlit.py`.
- Replace file paths with your own data for testing in Jupyter.

## Learning Objectives
- Build an interactive Streamlit dashboard for remote sensing data.
- Upload and process raster and vector data.
- Calculate and visualize spectral indices (e.g., NDVI, NDWI).
- Create interactive maps with Folium for geospatial visualization.

In [ ]:
# Import required libraries
import streamlit as st
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import plugins
from rasterio.warp import transform_bounds
from rasterio.mask import mask
import io
import os

# Streamlit page configuration
st.set_page_config(page_title='Remote Sensing Dashboard', layout='wide')

## Step 1: Create Streamlit Dashboard Layout

Set up the main dashboard interface with file uploaders and options.

In [ ]:
# Dashboard title and description
st.title('Interactive Remote Sensing Dashboard')
st.write('Upload a Sentinel-2 GeoTIFF and a GeoJSON/shapefile to calculate spectral indices and visualize results.')

# File uploaders
raster_file = st.file_uploader('Upload Sentinel-2 GeoTIFF (e.g., B02, B03, B04, B08)', type=['tif', 'tiff'])
vector_file = st.file_uploader('Upload GeoJSON or Shapefile (e.g., AOI)', type=['geojson', 'shp'])

# Index selection
index_options = ['NDVI (Vegetation)', 'NDWI (Water)']
selected_index = st.selectbox('Select Spectral Index', index_options)

# Band mapping (assuming Sentinel-2 band order)
band_map = {'B02': 0, 'B03': 1, 'B04': 2, 'B08': 3}  # Adjust based on your GeoTIFF band order

## Step 2: Process Uploaded Data

Load and preprocess the uploaded raster and vector files.

In [ ]:
def load_raster(file):
    with rasterio.open(file) as src:
        raster_data = src.read(masked=True)
        profile = src.profile
        bounds = src.bounds
        crs = src.crs
    return raster_data, profile, bounds, crs

def load_vector(file):
    if file.name.endswith('.geojson'):
        gdf = gpd.read_file(file)
    else:
        temp_dir = 'temp_vector'
        os.makedirs(temp_dir, exist_ok=True)
        temp_path = os.path.join(temp_dir, file.name)
        with open(temp_path, 'wb') as f:
            f.write(file.read())
        gdf = gpd.read_file(temp_path)
        os.remove(temp_path)
    return gdf

# Process files if uploaded
if raster_file and vector_file:
    try:
        # Load raster
        raster_data, raster_profile, raster_bounds, raster_crs = load_raster(raster_file)
        # Load vector
        vector_gdf = load_vector(vector_file)
        if vector_gdf.crs != raster_crs:
            vector_gdf = vector_gdf.to_crs(raster_crs)
        # Crop raster to vector extent
        with rasterio.open(raster_file) as src:
            cropped_data, cropped_transform = mask(src, vector_gdf.geometry, crop=True, nodata=np.nan)
        raster_profile.update({
            'height': cropped_data.shape[1],
            'width': cropped_data.shape[2],
            'transform': cropped_transform
        })
        raster_data = cropped_data
        st.success('Files loaded and aligned successfully!')
    except Exception as e:
        st.error(f'Error loading files: {str(e)}')
else:
    st.warning('Please upload both a raster and a vector file to proceed.')

## Step 3: Calculate Spectral Index

Compute the selected spectral index (NDVI or NDWI).

In [ ]:
def calculate_index(raster_data, index_type, band_map):
    red = raster_data[band_map['B04']].astype(float)
    nir = raster_data[band_map['B08']].astype(float)
    green = raster_data[band_map['B03']].astype(float)

    if index_type == 'NDVI (Vegetation)':
        index = np.where((nir + red) != 0, (nir - red) / (nir + red), np.nan)
    elif index_type == 'NDWI (Water)':
        index = np.where((green + nir) != 0, (green - nir) / (green + nir), np.nan)
    return index

if raster_file and vector_file:
    try:
        index_data = calculate_index(raster_data, selected_index, band_map)
        st.write(f'{selected_index} calculated successfully.')

        # Visualize index
        fig, ax = plt.subplots(figsize=(8, 8))
        im = ax.imshow(index_data, cmap='RdYlGn', vmin=-1, vmax=1)
        vector_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
        plt.colorbar(im, ax=ax, label=selected_index)
        plt.title(f'{selected_index} Map')
        plt.xlabel('Column')
        plt.ylabel('Row')
        st.pyplot(fig)
    except Exception as e:
        st.error(f'Error calculating index: {str(e)}')

## Step 4: Create Interactive Map with Folium

Display the index and vector data on an interactive Folium map.

In [ ]:
if raster_file and vector_file and 'index_data' in locals():
    try:
        # Save index as temporary GeoTIFF for mapping
        temp_index_path = 'temp_index.tif'
        index_profile = raster_profile.copy()
        index_profile.update({'count': 1, 'dtype': 'float32'})
        with rasterio.open(temp_index_path, 'w', **index_profile) as dst:
            dst.write(index_data, 1)

        # Convert bounds to WGS84
        bounds_latlon = transform_bounds(raster_crs, 'EPSG:4326', *raster_profile['bounds'])
        center_lat = (bounds_latlon[1] + bounds_latlon[3]) / 2
        center_lon = (bounds_latlon[0] + bounds_latlon[2]) / 2

        # Create Folium map
        m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

        # Add index layer
        folium.raster_layers.ImageOverlay(
            image=index_data,
            bounds=[[bounds_latlon[1], bounds_latlon[0]], [bounds_latlon[3], bounds_latlon[2]]],
            colormap=lambda x: (0, 1, 0, 0.5) if x > 0 else (1, 0, 0, 0.5),
            opacity=0.6
        ).add_to(m)

        # Add vector layer
        temp_geojson = 'temp_vector.geojson'
        vector_gdf.to_crs('EPSG:4326').to_file(temp_geojson)
        folium.GeoJson(temp_geojson, style_function=lambda x: {'color': 'red', 'weight': 2}).add_to(m)

        # Add map controls
        plugins.Fullscreen().add_to(m)
        folium.LayerControl().add_to(m)

        # Display map
        st.write('Interactive Map:')
        st_folium(m, width=700, height=500)

        # Clean up temporary files
        os.remove(temp_index_path)
        os.remove(temp_geojson)
    except Exception as e:
        st.error(f'Error creating map: {str(e)}')

## Step 5: Save Results

Allow users to download the calculated index as a GeoTIFF.

In [ ]:
if 'index_data' in locals():
    try:
        # Save index to temporary file for download
        output_index_path = f'{selected_index.lower()}_output.tif'
        with rasterio.open(output_index_path, 'w', **index_profile) as dst:
            dst.write(index_data, 1)

        # Provide download button
        with open(output_index_path, 'rb') as f:
            st.download_button(
                label=f'Download {selected_index} GeoTIFF',
                data=f,
                file_name=output_index_path,
                mime='image/tiff'
            )
        os.remove(output_index_path)
    except Exception as e:
        st.error(f'Error saving index: {str(e)}')

## Notes for Running as Streamlit App

To run this as a Streamlit app:
1. Save this notebook as `33_dashboard_streamlit.py`.
2. Run in terminal: `streamlit run 33_dashboard_streamlit.py`.
3. Open the provided URL in your browser to interact with the dashboard.

## Next Steps

- Test with your own Sentinel-2 GeoTIFF and GeoJSON/shapefile (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- Extend the dashboard by adding more indices (e.g., SAVI, EVI) or analysis options (e.g., classification from `12_classification_rf_svm.ipynb`).
- Integrate cloud detection (e.g., from `28_cloud_detection_deep_learning.ipynb`) as a preprocessing step.
- Explore advanced visualizations with `23_kepler_gl_demo.ipynb` or time series animations with `26_time_series_animation.ipynb`.

## Notes
- Ensure the GeoTIFF contains at least bands B02, B03, B04, and B08 for Sentinel-2.
- Adjust `band_map` if your GeoTIFF has a different band order.
- Streamlit requires a stable internet connection for Folium maps to render correctly.
- See `docs/installation.md` for troubleshooting library installation.